In [20]:
import instascrape
from bs4 import BeautifulSoup
from selenium import webdriver
import re
from webdriver_manager.chrome import ChromeDriverManager
import urllib.parse
from datastructures import Location, Site
from dataclasses_serialization.json import JSONSerializer
import time
import json
from tqdm import tqdm
from multiprocessing import Pool

driver = webdriver.Chrome(ChromeDriverManager().install())
base_url = "https://www.instagram.com"



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [/home/matas/.wdm/drivers/chromedriver/linux64/90.0.4430.24/chromedriver] found in cache


# 1. Get a list of all Locations that Instagram supports

In [5]:
driver.get(urllib.parse.urljoin(base_url, "/explore/locations/US/united-states/"))
soup = BeautifulSoup(driver.page_source)

data = {}

next_page = not None

while next_page is not None:
    next_page = soup.find("a", href=re.compile("\?page="))

    links = soup.find_all("a", href=re.compile("/explore/locations/"))
    for link in links:
        if link.text == "See More":
            continue
        data[link.text] = Location(link.text, link["href"], [])

    if next_page is None:
        break

    print("Getting the next page:", next_page["href"])
    driver.get(urllib.parse.urljoin(base_url, next_page["href"]))
    soup = BeautifulSoup(driver.page_source)
    time.sleep(2)

# Remove last link to more locations
del data["Locations"]

Getting the next page: /explore/locations/US/united-states/?page=2
Getting the next page: /explore/locations/US/united-states/?page=3
Getting the next page: /explore/locations/US/united-states/?page=4
Getting the next page: /explore/locations/US/united-states/?page=5
Getting the next page: /explore/locations/US/united-states/?page=6
Getting the next page: /explore/locations/US/united-states/?page=7
Getting the next page: /explore/locations/US/united-states/?page=8
Getting the next page: /explore/locations/US/united-states/?page=9
Getting the next page: /explore/locations/US/united-states/?page=10
Getting the next page: /explore/locations/US/united-states/?page=11


# Get the Sites from each Location

In [29]:
for key, location in data.items():
    print(location.name)

    driver.get(urllib.parse.urljoin(base_url, location.link))
    soup = BeautifulSoup(driver.page_source)

    next_page = not None

    while next_page is not None:
        next_page = soup.find("a", href=re.compile("\?page="))

        links = soup.find_all("a", href=re.compile("/explore/locations/[0-9]*/"))
        for link in links:
            if link.text == "See More":
                continue

            temp = instascrape.Location(urllib.parse.urljoin(base_url, link["href"]))
            try: 
                temp.scrape()
            except:
                print("Failed to scrape", link["href"])
                continue

            site = Site(link.text, link["href"], temp.amount_of_posts, temp.longitude, temp.latitude)
            if site not in location.sites:
                location.sites.append(site)
            time.sleep(0.1)
            
        if next_page is None:
            break

        print("Getting the next page:", next_page["href"])
        driver.get(urllib.parse.urljoin(base_url, next_page["href"]))
        soup = BeautifulSoup(driver.page_source)
        time.sleep(2)

    serialize_instagram_data(data, "checkpoint.json")

Midtown East
Failed to scrape /explore/locations/213724998/the-rink-at-rockefeller-center/


AttributeError: 'Location' object has no attribute 'amount_of_posts'

## Serialize / Deserialize the data

In [3]:
def serialize_instagram_data(data: dict, path: str) -> None:
    with open(path, "w") as file:
        json.dump(JSONSerializer.serialize(data), file)

def deserialize_instagram_data(path: str) -> dict:
    data = json.loads(open(path, "r").read())
    for key, val in data.items():
        data[key] = JSONSerializer.deserialize(Location, data[key])
    return data

In [6]:
data = deserialize_instagram_data("data.json")

In [22]:
def scrape_location(key: str) -> None:
    location = data[key]
    print(location.name)

    driver.get(urllib.parse.urljoin(base_url, location.link))
    soup = BeautifulSoup(driver.page_source)

    next_page = not None

    while next_page is not None:
        next_page = soup.find("a", href=re.compile("\?page="))

        links = soup.find_all("a", href=re.compile("/explore/locations/[0-9]*/"))
        for link in links:
            if link.text == "See More":
                continue

            temp = instascrape.Location(urllib.parse.urljoin(base_url, link["href"]))
            try: 
                temp.scrape()
            except:
                print("Failed to scrape", link["href"])
                continue

            site = Site(link.text, link["href"], temp.amount_of_posts, temp.longitude, temp.latitude)
            if site not in location.sites:
                location.sites.append(site)
            time.sleep(0.1)
            
        if next_page is None:
            break

        print("Getting the next page:", next_page["href"])
        driver.get(urllib.parse.urljoin(base_url, next_page["href"]))
        soup = BeautifulSoup(driver.page_source)
        time.sleep(2)

    serialize_instagram_data(data, f"checkpoint_{location.name}.json")

In [19]:
pool = Pool()
pool.map(scrape_location, list(data.keys()))

Midtown EastUpper West SideVictoriaWashington D.C.West HollywoodTacoma
Carson City
West Floral Heights Historic District

North Park Forest
Fig GardenPark SlopeFlatlands
Southfield
Panorama City

LenoxPalmetto I-75 Industrial Center
Greenwood Heights



Seguin
Downtown Long BeachOldhamElizabethtownWynwood
Truman Annex


Toms River
Gardena
Antioch
KearneyFranklin

North Hollywood WestMango Hill

HillcrestNew HavenNewtown
Mid-City WestKingsville



Douglas Byrd


Baldwin ParkWest Garden Grove
Showplace Square
QuincyVineyardMount Prospect
Los SerranosDeKalbGreen Hills


NameError: name 'base_url' is not defined